In [1]:
from  google.colab import drive  
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/algo trading/stocks
### save and load data using pickle
import pickle
def psave(dataframe, fname):
    with open('{}.pkl'.format(fname),'wb') as pickle_file:
        pickle.dump(dataframe,pickle_file)
        
def pload(fname):
  
  with open('{}.pkl'.format(fname),'rb') as pickle_file:
      dataframe=pickle.load(pickle_file)
      return dataframe

Mounted at /content/drive
/content/drive/My Drive/algo trading/stocks


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as mt
import datetime

from itertools import product
import sys

In [4]:
sys.path.append("/content/drive/MyDrive/Colab Notebooks/classes")
%reload_ext autoreload
%autoreload 2
from trading_main import account
from PropertyTest_DataGeneration import Gen as g
from DfPlot import dp

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
train_s=np.datetime64('2013-12-30')
train_e=np.datetime64('2017-12-29')

vali_s=np.datetime64('2018-01-02')
vali_e=np.datetime64('2018-12-31')

test_s=np.datetime64('2019-01-02')
test_e=np.datetime64('2019-12-30')
df=pload('selectedpairs_LSTM')

df = df.rename(columns={'return': 'ret'})

In [6]:
StockData=pload('StockPool')

In [8]:
#calculating the LSTM forecasted return on the validation set
length=len(df.iloc[0]['LSTM_vali'])

returnlstm=[]
for i in range(len(df)):
  r=[]
  sp=df.iloc[i]['spread'].loc[vali_s:vali_e]
  for j in range(length-1):
    r.append((df.iloc[i]['LSTM_vali'].iloc[j+1]-sp.iloc[j])/sp.iloc[j])


  inde=sp.iloc[:-1].index
  rr=pd.Series(r,index=inde)
  returnlstm.append(rr)


In [9]:
df['LSTM_vali_return']=returnlstm

In [30]:
#calculate the variance of the actual return on the training, validation set and the forecasted return on the validation set
v={'train_var':[],'vali_var':[],'LSTM_vali_var':[]}
for i in range(20):

  v['LSTM_vali_var'].append(df.iloc[i]['LSTM_vali_return'].var())
  v['vali_var'].append(df.iloc[i]['ret'].loc[vali_s:vali_e].var())
  v['train_var'].append(df.iloc[i]['ret'].loc[train_s:train_e].var())


In [31]:
d=pload('LSTM_result')
for i in v:
  d[i]=v[i]

In [32]:
d

,cross,mean1,pair,beta,spread,ret,mean2,cross2,mean3,cross3,meandiff,T,corr,LSTM_vali,LSTM_test,LSTM_mse,LSTM_para,para,vali_return,vali_record,LSTM_ret,LSTM_testreturn,LSTM_testtime,LSTM_losetime,train_var,vali_var,LSTM_vali_var
363082,37,-33.884243,"[CTHR, HAYN]",0.050000,Date 2013-12-02 2.929801 2013-12-03 2.79...,Date 2013-12-02 -0.044655 2013-12-03 -0.01...,-0.242134,40,-0.571331,18,0.329197,51.0,0.646790,Date 2018-01-02 -0.135967 2018-01-03 -0.21...,Date 2019-01-02 -0.386866 2019-01-03 -0.40...,0.002326,"(64, 32, 150)",(),0.000000,[],0.000000,0.000000,0.0,0,2.089608e+00,0.010816,0.000695
99348,33,-17.050050,"[AVNW, MMYT]",0.146939,Date 2013-12-02 11.808979 2013-12-03 11....,Date 2013-12-02 -0.025819 2013-12-03 -0.03...,3.632221,49,3.595694,26,0.036528,42.0,0.506716,Date 2018-01-02 3.195875 2018-01-03 3.00...,Date 2019-01-02 3.048844 2019-01-03 3.59...,0.056004,"(64, 64, 50)","(0.35, 0.05, 0.7, 0.85)",87.561959,"[NAN, Long, Clear, Long, Clear, Long, Clear, L...",0.000000,0.000000,0.0,0,6.182238e+00,0.005211,0.000139
288449,36,-27.130426,"[VOXX, NTCT]",0.224490,Date 2013-12-02 10.296735 2013-12-03 10....,Date 2013-12-02 0.029433 2013-12-03 -0.01...,-0.577103,57,-0.680304,28,0.103202,36.0,0.610608,Date 2018-01-02 -1.189168 2018-01-03 -1.42...,Date 2019-01-02 -1.229590 2019-01-03 -1.03...,0.038287,"(64, 64, 200)",(),0.000000,[],0.000000,0.000000,0.0,0,5.443592e+08,0.456656,0.043044
216234,35,-2.241410,"[TCBIL, VLGEA]",0.534694,Date 2013-12-02 1.446036 2013-12-03 1....,Date 2013-12-02 -0.154412 2013-12-03 1.13...,8.002588,53,8.540720,14,-0.538132,39.0,0.465233,Date 2018-01-02 9.703416 2018-01-03 9.63...,Date 2019-01-02 8.892055 2019-01-03 8....,0.062033,"(32, 32, 50)","(0.35, 0.05, 0.65, 1)",3.588377,"[NAN, Short, Clear, Short, Clear, Short, Clear...",-0.008139,-0.014239,3.0,0,1.321885e-03,0.000913,0.000110
314459,91,3.059907,"[FDUS, MGIC]",0.961224,Date 2013-12-02 4.596692 2013-12-03 4.48...,Date 2013-12-02 -0.023360 2013-12-03 0.00...,3.302077,91,2.671474,25,0.630603,23.0,0.871486,Date 2018-01-02 3.160225 2018-01-03 3.15...,Date 2019-01-02 2.210867 2019-01-03 2.30...,0.024078,"(16, 8, 150)","(0.35, 0, 0.65, 1)",4.589588,"[NAN, Short, Clear, Long, Clear, Short, Clear,...",0.000000,0.000000,0.0,0,2.803098e-03,0.004051,0.000720
363238,32,-6.491330,"[CTHR, CCLP]",0.185714,Date 2013-12-02 3.362121 2013-12-03 3.20...,Date 2013-12-02 -0.045473 2013-12-03 -0.00...,0.039106,34,0.189550,34,-0.150444,60.0,0.657552,Date 2018-01-02 0.489809 2018-01-03 0.38...,Date 2019-01-02 0.444297 2019-01-03 0.46...,0.002509,"(32, 16, 150)","(0.35, 0, 0.65, 0.95)",27.659026,"[NAN, Short, Clear, Long, Clear, Long, Clear, ...",-0.016352,1.038978,1.0,0,1.149110e+01,35.513797,3.029029
365248,38,-5.172277,"[TGA, CCLP]",0.282653,Date 2013-12-02 4.670289 2013-12-03 4.67...,Date 2013-12-02 0.001556 2013-12-03 0.06...,0.580727,39,0.724924,11,-0.144197,52.0,0.880470,Date 2018-01-02 0.140445 2018-01-03 0.11...,Date 2019-01-02 1.120790 2019-01-03 1.11...,0.013987,"(64, 64, 50)","(0.35, 0, 0.65, 1)",8.930240,"[NAN, Short, Clear, Short, Clear, Short, Clear...",0.000000,0.000000,0.0,0,6.935473e+00,3.490920,3.244509
288537,40,2.226178,"[VOXX, MARPS]",0.844898,Date 2013-12-02 9.853340 2013-12-03 10....,Date 2013-12-02 0.015442 2013-12-03 -0.00...,2.983573,48,3.017976,43,-0.034403,43.0,0.777196,Date 2018-01-02 3.501773 2018-01-03 3.47...,Date 2019-01-02 2.797970 2019-01-03 2.82...,0.027611,"(64, 64, 150)","(0.35, 0, 0.9, 1)",89.489938,"[NAN, Short, Clear, Short, Clear, Short, Clear...",0.000000,0.000000,0.0,0,2.305539e+00,0.003136,0.000068
439897,34,-1.855255,"[MIND, CCLP]",0.728571,Date 2013-12-02 9.905243 2013-12-03 9.67...,Date 2013-12-02 -0.023199 2013-12-03 -0.03...,0.321557,35,0.012027,15,0.309530,58.0,0.784027,Date 2018-01-02 0.007168 2018-01-03 0.02...,Date 2019-01-02 1.018617 2019-01-03 1.55...,0.027732,"(32, 16, 50)","(0.35, 0, 0.65, 0.8)",1.409449,"[NAN, Short, Clear, Short, Clear, Short, Clear...",1.276968,1.862351,3.0,0,3.150834e+00,174.453168,86.485471
9

In [33]:
d['tv_diff']=d['train_var']-d['vali_var']

In [34]:
#variance between acutual return of training data and validation data
d.sort_values('tv_diff')

,cross,mean1,pair,beta,spread,ret,mean2,cross2,mean3,cross3,meandiff,T,corr,LSTM_vali,LSTM_test,LSTM_mse,LSTM_para,para,vali_return,vali_record,LSTM_ret,LSTM_testreturn,LSTM_testtime,LSTM_losetime,train_var,vali_var,LSTM_vali_var,tv_diff
363116,32,-21.616234,"[CTHR, SWIR]",0.069388,Date 2013-12-02 3.838979 2013-12-03 3.70...,Date 2013-12-02 -0.033682 2013-12-03 -0.02...,-0.077458,42,-0.083219,21,0.005761,49.0,0.097329,Date 2018-01-02 -0.079678 2018-01-03 -0.15...,Date 2019-01-02 -0.082213 2019-01-03 -0.08...,0.002881,"(32, 16, 200)",(),0.000000,[],0.000000,0.000000,0.0,0,1.598158e+02,381.777443,2.775072,-2.219616e+02
439897,34,-1.855255,"[MIND, CCLP]",0.728571,Date 2013-12-02 9.905243 2013-12-03 9.67...,Date 2013-12-02 -0.023199 2013-12-03 -0.03...,0.321557,35,0.012027,15,0.309530,58.0,0.784027,Date 2018-01-02 0.007168 2018-01-03 0.02...,Date 2019-01-02 1.018617 2019-01-03 1.55...,0.027732,"(32, 16, 50)","(0.35, 0, 0.65, 0.8)",1.409449,"[NAN, Short, Clear, Short, Clear, Short, Clear...",1.276968,1.862351,3.0,0,3.150834e+00,174.453168,86.485471,-1.713023e+02
363238,32,-6.491330,"[CTHR, CCLP]",0.185714,Date 2013-12-02 3.362121 2013-12-03 3.20...,Date 2013-12-02 -0.045473 2013-12-03 -0.00...,0.039106,34,0.189550,34,-0.150444,60.0,0.657552,Date 2018-01-02 0.489809 2018-01-03 0.38...,Date 2019-01-02 0.444297 2019-01-03 0.46...,0.002509,"(32, 16, 150)","(0.35, 0, 0.65, 0.95)",27.659026,"[NAN, Short, Clear, Long, Clear, Long, Clear, ...",-0.016352,1.038978,1.0,0,1.149110e+01,35.513797,3.029029,-2.402270e+01
314459,91,3.059907,"[FDUS, MGIC]",0.961224,Date 2013-12-02 4.596692 2013-12-03 4.48...,Date 2013-12-02 -0.023360 2013-12-03 0.00...,3.302077,91,2.671474,25,0.630603,23.0,0.871486,Date 2018-01-02 3.160225 2018-01-03 3.15...,Date 2019-01-02 2.210867 2019-01-03 2.30...,0.024078,"(16, 8, 150)","(0.35, 0, 0.65, 1)",4.589588,"[NAN, Short, Clear, Long, Clear, Short, Clear,...",0.000000,0.000000,0.0,0,2.803098e-03,0.004051,0.000720,-1.247848e-03
343879,53,10.455164,"[PBPB, ZNGA]",1.000000,Date 2013-12-02 23.96 2013-12-03 24.18 2...,Date 2013-12-02 0.009182 2013-12-03 -0.02...,10.455164,53,8.369160,35,2.086004,39.0,0.517522,Date 2018-01-02 8.284303 2018-01-03 8.69...,Date 2019-01-02 4.032262 2019-01-03 3.95...,0.061774,"(32, 32, 150)","(0.35, 0, 0.65, 1)",120.330605,"[NAN, Long, Clear, Short, Clear, Short, Clear,...",11.800812,1.059845,3.0,1,9.527355e-04,0.000935,0.000048,1.777051e-05
215806,71,15.386518,"[TCBIL, ZIXI]",1.000000,Date 2013-12-02 12.242821 2013-12-03 12....,Date 2013-12-02 0.008645 2013-12-03 -0.02...,15.386518,71,16.092516,35,-0.705998,29.0,0.336621,Date 2018-01-02 16.166687 2018-01-03 16....,Date 2019-01-02 16.078003 2019-01-03 16....,0.027739,"(16, 8, 100)","(0.35, 0.05, 0.9, 1)",7.162815,"[NAN, Short, Clear, Short, Clear, Short, Clear...",0.425220,-0.012719,9.0,5,1.455704e-04,0.000111,0.000011,3.501145e-05
374562,45,-73.893814,"[DSPG, NXPI]",0.069388,Date 2013-12-02 5.870144 2013-12-03 5.85...,Date 2013-12-02 -0.003418 2013-12-03 -0.00...,4.618592,50,5.405373,15,-0.786781,41.0,0.787361,Date 2018-01-02 4.730648 2018-01-03 4.49...,Date 2019-01-02 6.290832 2019-01-03 6.41...,0.055822,"(32, 32, 150)","(0.35, 0.1, 0.65, 1)",13.796608,"[NAN, Long, Clear, Long, Clear, Long, Clear, S...",-0.047578,0.150801,3.0,2,2.403588e-03,0.002054,0.000100,3.493411e-04
216234,35,-2.241410,"[TCBIL, VLGEA]",0.534694,Date 2013-12-02 1.446036 2013-12-03 1....,Date 2013-12-02 -0.154412 2013-12-03 1.13...,8.002588,53,8.540720,14,-0.538132,39.0,0.465233,Date 2018-01-02 9.703416 2018-01-03 9.63...,Date 2019-01-02 8.892055 2019-01-03 8....,0.062033,"(32, 32, 50)","(0.35, 0.05, 0.65, 1)",3.588377,"[NAN, Short, Clear, Short, Clear, Short, Clear...",-0.008139,-0.014239,3.0,0,1.321885e-03,0.000913,0.000110,4.093519e-04
343984,40,8.920654,"[PBPB, SREV]",1.000000,Date 2013-12-02 18.740000 2013-12-03 19....,Date 2013-12-02 5.389542e-02 2013-12-03 ...,8.920654,40,9.105960,33,-0.185306,51.0,0.763349,Date 2018-01-02 9.227072 2018-01-03 9.49...,Date 2019-01-02 7

In [36]:
d['tv_diff_abs']=abs(d['train_var']-d['vali_var'])

In [38]:
#sort the difference's absolute value
d.sort_values('tv_diff_abs')

,cross,mean1,pair,beta,spread,ret,mean2,cross2,mean3,cross3,meandiff,T,corr,LSTM_vali,LSTM_test,LSTM_mse,LSTM_para,para,vali_return,vali_record,LSTM_ret,LSTM_testreturn,LSTM_testtime,LSTM_losetime,train_var,vali_var,LSTM_vali_var,tv_diff,tv_diff_abs
343879,53,10.455164,"[PBPB, ZNGA]",1.000000,Date 2013-12-02 23.96 2013-12-03 24.18 2...,Date 2013-12-02 0.009182 2013-12-03 -0.02...,10.455164,53,8.369160,35,2.086004,39.0,0.517522,Date 2018-01-02 8.284303 2018-01-03 8.69...,Date 2019-01-02 4.032262 2019-01-03 3.95...,0.061774,"(32, 32, 150)","(0.35, 0, 0.65, 1)",120.330605,"[NAN, Long, Clear, Short, Clear, Short, Clear,...",11.800812,1.059845,3.0,1,9.527355e-04,0.000935,0.000048,1.777051e-05,1.777051e-05
215806,71,15.386518,"[TCBIL, ZIXI]",1.000000,Date 2013-12-02 12.242821 2013-12-03 12....,Date 2013-12-02 0.008645 2013-12-03 -0.02...,15.386518,71,16.092516,35,-0.705998,29.0,0.336621,Date 2018-01-02 16.166687 2018-01-03 16....,Date 2019-01-02 16.078003 2019-01-03 16....,0.027739,"(16, 8, 100)","(0.35, 0.05, 0.9, 1)",7.162815,"[NAN, Short, Clear, Short, Clear, Short, Clear...",0.425220,-0.012719,9.0,5,1.455704e-04,0.000111,0.000011,3.501145e-05,3.501145e-05
374562,45,-73.893814,"[DSPG, NXPI]",0.069388,Date 2013-12-02 5.870144 2013-12-03 5.85...,Date 2013-12-02 -0.003418 2013-12-03 -0.00...,4.618592,50,5.405373,15,-0.786781,41.0,0.787361,Date 2018-01-02 4.730648 2018-01-03 4.49...,Date 2019-01-02 6.290832 2019-01-03 6.41...,0.055822,"(32, 32, 150)","(0.35, 0.1, 0.65, 1)",13.796608,"[NAN, Long, Clear, Long, Clear, Long, Clear, S...",-0.047578,0.150801,3.0,2,2.403588e-03,0.002054,0.000100,3.493411e-04,3.493411e-04
216234,35,-2.241410,"[TCBIL, VLGEA]",0.534694,Date 2013-12-02 1.446036 2013-12-03 1....,Date 2013-12-02 -0.154412 2013-12-03 1.13...,8.002588,53,8.540720,14,-0.538132,39.0,0.465233,Date 2018-01-02 9.703416 2018-01-03 9.63...,Date 2019-01-02 8.892055 2019-01-03 8....,0.062033,"(32, 32, 50)","(0.35, 0.05, 0.65, 1)",3.588377,"[NAN, Short, Clear, Short, Clear, Short, Clear...",-0.008139,-0.014239,3.0,0,1.321885e-03,0.000913,0.000110,4.093519e-04,4.093519e-04
343984,40,8.920654,"[PBPB, SREV]",1.000000,Date 2013-12-02 18.740000 2013-12-03 19....,Date 2013-12-02 5.389542e-02 2013-12-03 ...,8.920654,40,9.105960,33,-0.185306,51.0,0.763349,Date 2018-01-02 9.227072 2018-01-03 9.49...,Date 2019-01-02 7.041347 2019-01-03 7.03...,0.062894,"(64, 32, 50)","(0.35, 0.2, 0.8, 1)",560.928311,"[NAN, Short, Clear, Short, Clear, Short, Clear...",0.750184,0.687607,2.0,1,1.422584e-03,0.000823,0.000018,5.999614e-04,5.999614e-04
370078,40,-8.746139,"[DAKT, EPZM]",0.166327,Date 2013-12-02 8.709031 2013-12-03 8.58...,Date 2013-12-02 -0.014439 2013-12-03 -0.05...,5.959125,58,5.887973,16,0.071153,35.0,0.705655,Date 2018-01-02 6.390990 2018-01-03 6.40...,Date 2019-01-02 6.093053 2019-01-03 5.97...,0.037666,"(32, 16, 100)","(0.35, 0.2, 0.9, 0.9)",108.183795,"[NAN, Long, Clear, Short, Clear, Short, Clear,...",-0.321576,-0.125911,1.0,0,1.707986e-03,0.000997,0.000026,7.108550e-04,7.108550e-04
370108,33,-4.390121,"[DAKT, GSM]",0.360204,Date 2013-12-02 6.418404 2013-12-03 6.48...,Date 2013-12-02 0.009817 2013-12-03 -0.02...,4.108352,49,4.596163,19,-0.487811,42.0,0.742318,Date 2018-01-02 2.753217 2018-01-03 2.73...,Date 2019-01-02 6.411877 2019-01-03 6.34...,0.044052,"(64, 64, 200)","(0.35, 0, 0.8, 1)",76.281659,"[NAN, Short, Clear, Long, Clear, Short, Clear,...",0.000000,0.798693,1.0,0,2.926485e-03,0.001919,0.000190,1.007719e-03,1.007719e-03
299691,47,2.282393,"[CWCO, PFLT]",0.786735,Date 2013-12-02 4.630280 2013-12-03 4.82...,Date 2013-12-02 0.041284 2013-12-03 -0.02...,3.996177,63,4.600242,26,-0.604065,33.0,0.362194,Date 2018-01-02 3.914666 2018-01-03 3.95...,Date 2019-01-02 3.937359 2019-01-03 3.65...,0.059476,"(64, 64, 100)","(0.35, 0.05, 0.65, 1)",11.321846,"[NAN, Long, Clear, Long, Clear, Long, Clear, S...",0.832100,0.013660,2.0,1,3.856408e-03,0.002822,0.000162,1.034079e-03,1.034079e-03
314459,91,3.059907,"[FDUS, MGIC]",0.961224,Date 2013-12-02 4.596692 2

In [50]:
#mean mse on the validation set
d['LSTM_mse'].mean()

0.035336614889092746

In [39]:
d['LSTM_mse'].describe()

count    20.000000
mean      0.035337
std       0.021585
min       0.002326
25%       0.021556
50%       0.037977
75%       0.055867
max       0.062894
Name: LSTM_mse, dtype: float64

In [43]:
newd=d.loc[d['para']!=()]

In [47]:
#mean rate of return using the LSTM forecast
d['LSTM_testreturn'].sum()/16

0.3307940128691494

In [49]:
d['LSTM_testreturn'].describe()

count    20.000000
mean      0.264635
std       0.535478
min      -0.166363
25%       0.000000
50%       0.000000
75%       0.285002
max       1.862351
Name: LSTM_testreturn, dtype: float64

In [53]:
#number of profitable pair
len(d.index[d['LSTM_testreturn']>0])

7

In [54]:
#number of unprofitable pair
len(d.index[d['LSTM_testreturn']<0])

4

In [58]:
#total number of trades
d['LSTM_testtime'].sum()

33.0

In [59]:
#total number of unprofitable trades
d['LSTM_losetime'].sum()

12

In [63]:
#difference between variance of the actural return and LSTM forecasted return on the validation set
d['vv_diff']=d['vali_var']-d['LSTM_vali_var']

In [64]:
d['vv_diff'].describe()

count     20.000000
mean      25.007780
std       85.817684
min        0.000099
25%        0.000950
50%        0.002864
75%        0.069193
max      379.002371
Name: vv_diff, dtype: float64

In [66]:
d.sort_values('vv_diff')

,cross,mean1,pair,beta,spread,ret,mean2,cross2,mean3,cross3,meandiff,T,corr,LSTM_vali,LSTM_test,LSTM_mse,LSTM_para,para,vali_return,vali_record,LSTM_ret,LSTM_testreturn,LSTM_testtime,LSTM_losetime,train_var,vali_var,LSTM_vali_var,tv_diff,tv_diff_abs,vv_diff_abs,vv_diff
215806,71,15.386518,"[TCBIL, ZIXI]",1.000000,Date 2013-12-02 12.242821 2013-12-03 12....,Date 2013-12-02 0.008645 2013-12-03 -0.02...,15.386518,71,16.092516,35,-0.705998,29.0,0.336621,Date 2018-01-02 16.166687 2018-01-03 16....,Date 2019-01-02 16.078003 2019-01-03 16....,0.027739,"(16, 8, 100)","(0.35, 0.05, 0.9, 1)",7.162815,"[NAN, Short, Clear, Short, Clear, Short, Clear...",0.425220,-0.012719,9.0,5,1.455704e-04,0.000111,0.000011,3.501145e-05,3.501145e-05,0.000099,0.000099
213032,36,-22.203382,"[BPOPN, CASS]",0.243878,Date 2013-12-02 1.947397 2013-12-03 1....,Date 2013-12-02 -0.041767 2013-12-03 0.10...,6.451466,37,8.476603,25,-2.025137,55.0,0.760750,Date 2018-01-02 8.481344 2018-01-03 8.45...,Date 2019-01-02 9.508059 2019-01-03 9....,0.050938,"(32, 16, 150)","(0.35, 0.05, 0.75, 0.85)",3.409375,"[NAN, Long, Clear, Long, Clear, Short, Clear, ...",-0.252854,-0.166363,5.0,2,3.455446e-03,0.000730,0.000084,2.725847e-03,2.725847e-03,0.000645,0.000645
216234,35,-2.241410,"[TCBIL, VLGEA]",0.534694,Date 2013-12-02 1.446036 2013-12-03 1....,Date 2013-12-02 -0.154412 2013-12-03 1.13...,8.002588,53,8.540720,14,-0.538132,39.0,0.465233,Date 2018-01-02 9.703416 2018-01-03 9.63...,Date 2019-01-02 8.892055 2019-01-03 8....,0.062033,"(32, 32, 50)","(0.35, 0.05, 0.65, 1)",3.588377,"[NAN, Short, Clear, Short, Clear, Short, Clear...",-0.008139,-0.014239,3.0,0,1.321885e-03,0.000913,0.000110,4.093519e-04,4.093519e-04,0.000802,0.000802
343984,40,8.920654,"[PBPB, SREV]",1.000000,Date 2013-12-02 18.740000 2013-12-03 19....,Date 2013-12-02 5.389542e-02 2013-12-03 ...,8.920654,40,9.105960,33,-0.185306,51.0,0.763349,Date 2018-01-02 9.227072 2018-01-03 9.49...,Date 2019-01-02 7.041347 2019-01-03 7.03...,0.062894,"(64, 32, 50)","(0.35, 0.2, 0.8, 1)",560.928311,"[NAN, Short, Clear, Short, Clear, Short, Clear...",0.750184,0.687607,2.0,1,1.422584e-03,0.000823,0.000018,5.999614e-04,5.999614e-04,0.000804,0.000804
343879,53,10.455164,"[PBPB, ZNGA]",1.000000,Date 2013-12-02 23.96 2013-12-03 24.18 2...,Date 2013-12-02 0.009182 2013-12-03 -0.02...,10.455164,53,8.369160,35,2.086004,39.0,0.517522,Date 2018-01-02 8.284303 2018-01-03 8.69...,Date 2019-01-02 4.032262 2019-01-03 3.95...,0.061774,"(32, 32, 150)","(0.35, 0, 0.65, 1)",120.330605,"[NAN, Long, Clear, Short, Clear, Short, Clear,...",11.800812,1.059845,3.0,1,9.527355e-04,0.000935,0.000048,1.777051e-05,1.777051e-05,0.000887,0.000887
370078,40,-8.746139,"[DAKT, EPZM]",0.166327,Date 2013-12-02 8.709031 2013-12-03 8.58...,Date 2013-12-02 -0.014439 2013-12-03 -0.05...,5.959125,58,5.887973,16,0.071153,35.0,0.705655,Date 2018-01-02 6.390990 2018-01-03 6.40...,Date 2019-01-02 6.093053 2019-01-03 5.97...,0.037666,"(32, 16, 100)","(0.35, 0.2, 0.9, 0.9)",108.183795,"[NAN, Long, Clear, Short, Clear, Short, Clear,...",-0.321576,-0.125911,1.0,0,1.707986e-03,0.000997,0.000026,7.108550e-04,7.108550e-04,0.000971,0.000971
99073,34,-10.444400,"[AVNW, EPZM]",0.146939,Date 2013-12-02 11.102204 2013-12-03 10....,Date 2013-12-02 -0.028125 2013-12-03 -0.05...,4.602847,44,6.254098,32,-1.651251,46.0,0.696870,Date 2018-01-02 5.712224 2018-01-03 5.71...,Date 2019-01-02 5.678904 2019-01-03 6.12...,0.043209,"(32, 16, 50)","(0.35, 0.05, 0.8, 1)",1371.746005,"[NAN, Long, Clear, Long, Clear, Long, Clear, S...",0.000000,0.000000,0.0,0,4.481985e-03,0.001211,0.000040,3.270887e-03,3.270887e-03,0.001171,0.001171
370108,33,-4.390121,"[DAKT, GSM]",0.360204,Date 2013-12-02 6.418404 2013-12-03 6.48...,Date 2013-12-02 0.009817 2013-12-03 -0.02...,4.108352,49,4.596163,19,-0.487811,42.0,0.742318,Date 2018-01-02 2.753217 2018-01-03 2.73...,Date 2019-01-02 6.411877 2019-01-03 6.34...,0.044052,"(64, 64, 200)","(0.35, 0, 0.8, 1)",76.281659,"[NAN, Short, Clear, Long, Clear, Short, Clear,...",0.

In [69]:
num=[]
for i in range(20):
  n=len(d.iloc[i]['vali_record'])
  num.append(n)

In [71]:
d['vali_time']=num

In [73]:
#total number of trades on the validation set
d['vali_time'].sum()

1458

In [72]:
d.sort_values('vali_time')

,cross,mean1,pair,beta,spread,ret,mean2,cross2,mean3,cross3,meandiff,T,corr,LSTM_vali,LSTM_test,LSTM_mse,LSTM_para,para,vali_return,vali_record,LSTM_ret,LSTM_testreturn,LSTM_testtime,LSTM_losetime,train_var,vali_var,LSTM_vali_var,tv_diff,tv_diff_abs,vv_diff_abs,vv_diff,vali_time
363082,37,-33.884243,"[CTHR, HAYN]",0.050000,Date 2013-12-02 2.929801 2013-12-03 2.79...,Date 2013-12-02 -0.044655 2013-12-03 -0.01...,-0.242134,40,-0.571331,18,0.329197,51.0,0.646790,Date 2018-01-02 -0.135967 2018-01-03 -0.21...,Date 2019-01-02 -0.386866 2019-01-03 -0.40...,0.002326,"(64, 32, 150)",(),0.000000,[],0.000000,0.000000,0.0,0,2.089608e+00,0.010816,0.000695,2.078792e+00,2.078792e+00,0.010121,0.010121,0
403782,34,-19.360815,"[FTEK, GPRE]",0.127551,Date 2013-12-02 5.641217 2013-12-03 5.76...,Date 2013-12-02 0.022051 2013-12-03 -0.00...,-0.237855,43,-0.996922,24,0.759067,47.0,0.457218,Date 2018-01-02 -0.926618 2018-01-03 -0.93...,Date 2019-01-02 -0.415494 2019-01-03 -0.39...,0.005712,"(64, 64, 200)",(),0.000000,[],0.000000,0.000000,0.0,0,1.852614e+00,0.007802,0.000367,1.844811e+00,1.844811e+00,0.007436,0.007436,0
288449,36,-27.130426,"[VOXX, NTCT]",0.224490,Date 2013-12-02 10.296735 2013-12-03 10....,Date 2013-12-02 0.029433 2013-12-03 -0.01...,-0.577103,57,-0.680304,28,0.103202,36.0,0.610608,Date 2018-01-02 -1.189168 2018-01-03 -1.42...,Date 2019-01-02 -1.229590 2019-01-03 -1.03...,0.038287,"(64, 64, 200)",(),0.000000,[],0.000000,0.000000,0.0,0,5.443592e+08,0.456656,0.043044,5.443592e+08,5.443592e+08,0.413612,0.413612,0
363116,32,-21.616234,"[CTHR, SWIR]",0.069388,Date 2013-12-02 3.838979 2013-12-03 3.70...,Date 2013-12-02 -0.033682 2013-12-03 -0.02...,-0.077458,42,-0.083219,21,0.005761,49.0,0.097329,Date 2018-01-02 -0.079678 2018-01-03 -0.15...,Date 2019-01-02 -0.082213 2019-01-03 -0.08...,0.002881,"(32, 16, 200)",(),0.000000,[],0.000000,0.000000,0.0,0,1.598158e+02,381.777443,2.775072,-2.219616e+02,2.219616e+02,379.002371,379.002371,0
288537,40,2.226178,"[VOXX, MARPS]",0.844898,Date 2013-12-02 9.853340 2013-12-03 10....,Date 2013-12-02 0.015442 2013-12-03 -0.00...,2.983573,48,3.017976,43,-0.034403,43.0,0.777196,Date 2018-01-02 3.501773 2018-01-03 3.47...,Date 2019-01-02 2.797970 2019-01-03 2.82...,0.027611,"(64, 64, 150)","(0.35, 0, 0.9, 1)",89.489938,"[NAN, Short, Clear, Short, Clear, Short, Clear...",0.000000,0.000000,0.0,0,2.305539e+00,0.003136,0.000068,2.302402e+00,2.302402e+00,0.003068,0.003068,67
370108,33,-4.390121,"[DAKT, GSM]",0.360204,Date 2013-12-02 6.418404 2013-12-03 6.48...,Date 2013-12-02 0.009817 2013-12-03 -0.02...,4.108352,49,4.596163,19,-0.487811,42.0,0.742318,Date 2018-01-02 2.753217 2018-01-03 2.73...,Date 2019-01-02 6.411877 2019-01-03 6.34...,0.044052,"(64, 64, 200)","(0.35, 0, 0.8, 1)",76.281659,"[NAN, Short, Clear, Long, Clear, Short, Clear,...",0.000000,0.798693,1.0,0,2.926485e-03,0.001919,0.000190,1.007719e-03,1.007719e-03,0.001729,0.001729,73
215806,71,15.386518,"[TCBIL, ZIXI]",1.000000,Date 2013-12-02 12.242821 2013-12-03 12....,Date 2013-12-02 0.008645 2013-12-03 -0.02...,15.386518,71,16.092516,35,-0.705998,29.0,0.336621,Date 2018-01-02 16.166687 2018-01-03 16....,Date 2019-01-02 16.078003 2019-01-03 16....,0.027739,"(16, 8, 100)","(0.35, 0.05, 0.9, 1)",7.162815,"[NAN, Short, Clear, Short, Clear, Short, Clear...",0.425220,-0.012719,9.0,5,1.455704e-04,0.000111,0.000011,3.501145e-05,3.501145e-05,0.000099,0.000099,77
370078,40,-8.746139,"[DAKT, EPZM]",0.166327,Date 2013-12-02 8.709031 2013-12-03 8.58...,Date 2013-12-02 -0.014439 2013-12-03 -0.05...,5.959125,58,5.887973,16,0.071153,35.0,0.705655,Date 2018-01-02 6.390990 2018-01-03 6.40...,Date 2019-01-02 6.093053 2019-01-03 5.97...,0.037666,"(32, 16, 100)","(0.35, 0.2, 0.9, 0.9)",108.183795,"[NAN, Long, Clear, Short, Clear, Short, Clear,...",-0.321576,-0.125911,1.0,0,1.707986e-03,0.000997,0.000026,7.108550e-04,7.108550e-04,0.000971,0.000971,79
439897,34,-1.855255,"[MIND, CCLP]",0.728571,Date 2013-12-02 9.905243 2013-12-03 9.67...,Date 2013-12-02 -0.023199 2013-12-03 